# Optimizing Ridge regression with linalg.solve

Author: Alexandre Gramfort

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
linalg.solve?

When using `assume_a='pos'` the system is solved with `Cholesky` factorization (\*POSV functions in Lapack)

When using `assume_a='gen'` the system is solved with `standard Gauss pivoting` (LU) factorization (\*GESV functions in Lapack)

When using `assume_a='sym'` the system is solved with `diagonal pivoting` factorization (\*SYSV functions in Lapack)

## Generate simulated data

$y = Xw+b+noise$

In [ ]:
np.random.seed(0)
n_samples, n_features = 100, 1
X = np.random.randn(n_samples, n_features)
w = np.random.randn(n_features)
b = 10.
y = np.dot(X, w) + b
y += 0.3 * np.random.randn(n_samples)

In [ ]:
def plot_data(w, b=0.):
    plt.plot(X[:, 0], y, 'o', alpha=0.2)

    xx = np.linspace(-2, 2, 100)
    yy = np.dot(xx[:, np.newaxis], w) + b
    plt.plot(xx, yy, 'k')
    plt.grid(True)

plot_data(w, b)

## Option 1 : center X and y

$\min_w \frac{1}{2} \|y - Xw - b\|^2 + \frac{\alpha}{2} \|w\|^2$

solved with `linalg.solve`

In [ ]:
X_mean = np.mean(X, axis=0)
y_mean = np.mean(y)

X_centered = X - X_mean
y_centered = y - y_mean

alpha = 100.
w_hat = linalg.solve(np.dot(X_centered.T, X_centered) + alpha * np.eye(1),
                     np.dot(X_centered.T, y), assume_a='pos')
b_hat = - np.dot(X_mean, w_hat) + y_mean

plot_data(w_hat, b_hat)

## Option 2: add a column with ones

In [ ]:
Xb = np.concatenate((X, np.ones((n_samples, 1))), axis=1)

alpha = 100.

# solve in dimension p + 1:
w_hat, b_hat = linalg.solve(np.dot(Xb.T, Xb) + alpha * np.eye(2), np.dot(Xb.T, y), assume_a='pos')
# w_hat, b_hat = linalg.solve(np.dot(Xb.T, Xb) + np.diag([alpha, 0.]), np.dot(Xb.T, y), assume_a='pos')
plot_data(w_hat, b_hat)

**Take home message:** Pay attention not to regularize the intercept...